In [46]:
%cd /home/slav/ai/claim-rank
!export PYTHONPATH=.
%load_ext autoreload
%autoreload 2

/home/slav/ai/claim-rank
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from tempfile import NamedTemporaryFile
import numpy as np
from src.models.keras.model_runner import *


In [48]:
from keras.layers import Input, Embedding, Conv1D, Dense
from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout
from keras import optimizers
from sklearn.metrics import average_precision_score

In [49]:
def load_crossvalidation_data(name='crossval'):
    folds = np.load('folds/' + name + '.npy')
    return folds;

In [57]:
def create_baseline_model(in_count, out_count):
    input_layer = Input(shape=(in_count,))
    x = Dense(1000, kernel_initializer='normal', activation='relu')(input_layer)
    
    outputs = list(map(lambda i: Dense(1, kernel_initializer='normal', activation='sigmoid', name='pred_' + str(i))(x), range(out_count)))

    model = Model(inputs=[input_layer], outputs=outputs)
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy',
                  metrics=['accuracy'],
                  loss_weights=loss_weights)
    return model;

In [51]:
def run_model(X_train, X_test, train_target, test_targets, create_model = create_baseline_model, epochs=15):
    model = create_model(X_train.shape[1], 1)
    model.fit(X_train, train_target, epochs=epochs, batch_size=550, verbose=0)
    return [average_precision_score(test_targets[j], model.predict(X_test)[:, 0])
         for j in range(len(test_targets))]
    

In [52]:
def run_model_multiple(X_train, X_test, train_targets, test_targets, log_indexes, create_model = create_baseline_model, epochs=20):
    model = create_model(X_train.shape[1], len(train_targets))
    
    
    model.fit(X_train, train_targets, epochs=epochs, batch_size=550, verbose=0)
    return [average_precision_score(test_targets[j], model.predict(X_test)[i][:, 0])
            for j, i in enumerate(log_indexes)]

In [53]:
def cross_validate(folds, train_targets, test_targets):
    models_sums = [0 for _ in test_targets]
    for fold in folds:
        X_train, X_test, y_train, y_test = fold
        y_train = list(y_train[:, train_targets].T)
        y_test = y_test[:, test_targets].T
        models = run_model(X_train, X_test, y_train, y_test)
        models_sums = [prev + curr for prev, curr in zip(models_sums, models)]

    print(np.array(models_sums) / len(folds))

In [54]:
def cross_validate_multiple(folds, train_targets, test_targets, log_indexes):
    models_sums = [0 for _ in test_targets]
    for fold in folds:
        X_train, X_test, y_train, y_test = fold
        y_train = list(y_train[:, train_targets].T)
        y_test = y_test[:, test_targets].T
        models = run_model_multiple(X_train, X_test, y_train, y_test, log_indexes)
        models_sums = [prev + curr for prev, curr in zip(models_sums, models)]

    print(np.array(models_sums) / len(folds))

In [55]:
folds = load_crossvalidation_data(name='serialized')


In [12]:
init_session(use_gpu=False)
cross_validate(folds, [6], [6])

[0.1889362]


In [16]:
for i in range(1,10):
    cross_validate(folds, [i], [i])

[ 0.07586703]
[ 0.06432117]
[ 0.06344388]
[ 0.09402855]
[ 0.16960528]
[ 0.21435139]
[ 0.14386683]
[ 0.13278658]
[ 0.07033692]


In [14]:
cross_validate_multiple(folds, range(10), range(1, 10), range(1, 10))

[0.09046304 0.10566049 0.0910099  0.07913272 0.17872785 0.20270445
 0.13284969 0.15193886 0.10933569]


In [16]:
cross_validate_multiple(folds, [0, 6], [0, 6], [0, 1])

[0.36771186 0.18494271]


In [13]:
init_session(use_gpu=False)
cross_validate_multiple(folds, range(10), [0, 6], [0, 6])

[0.32238642 0.16125607]


In [12]:
folds = load_crossvalidation_data(name='serialized')
init_session(use_gpu=False)
cross_validate(folds, [6], [6])
cross_validate_multiple(folds, range(10), [0, 6], [0, 6])

[0.18784728]
[0.37066138 0.19792223]


In [11]:
folds = load_crossvalidation_data(name='serialized')
# init_session(use_gpu=False)
cross_validate(folds, [6], [6])
cross_validate_multiple(folds, [0, 6], [0, 6], [0, 1])
cross_validate_multiple(folds, range(10), [0, 6], [0, 6])

[0.18944541]
[0.31675787 0.14813076]
[0.32264965 0.15744599]


In [58]:
folds = load_crossvalidation_data(name='serialized')
init_session(use_gpu=False)
# cross_validate(folds, [6], [6])
cross_validate_multiple(folds, range(10), [0, 6], [0, 6])

[0.38592118 0.19553073]
